In [3]:
import pandas as pd
import os
# import seaborn as sns
import numpy as np

cwd = os.getcwd()
print(cwd)
os.chdir("C:\\Users\\Administrator\\Desktop\\Python directory")

/Users/bruceyu/Desktop/Running_Proj/sosc4300_proj/sosc4300_vis_data_exploration


In [5]:
weibo_df = pd.read_csv("SexWeibo.csv")

In [6]:
#Changing Data Types
#The data types of each column of the data set need to be as following so that SnowNLP can be properly used to analyze the data.

#Gender: binary (Male --> 1; Female --> 0)
#Province: integer
#WeiboID: integer
#Keyword: integer
#Text: unicode string

gender = {'女': 1,'男': 0}

weibo_df.user_gender = [gender[item] for item in weibo_df.user_gender]
# change data type of Province, City, Keyword and WeiboID from string to integer
weibo_df['user_location'] = pd.to_numeric(weibo_df['user_location'], errors='ignore')
weibo_df['weibo_id'] = pd.to_numeric(weibo_df['weibo_id'], errors='ignore')
weibo_df['keyword'] = pd.to_numeric(weibo_df['keyword'], errors='ignore')
# change data type of Text from str to unicode string to preserve special characters (Chinese characters)
weibo_df['weibo_text'] = weibo_df['weibo_text'].astype('unicode')

In [7]:
# import necessary library
from snownlp import SnowNLP

# adding sentiment for each row of each dataset
weibo_df['Sentiment'] = SnowNLP(weibo_df['weibo_text'])
for row in range(len(weibo_df)):
    weibo_df['Sentiment'][row] = SnowNLP(weibo_df['weibo_text'][row]).sentiments

<ipython-input-7-795b594ee474>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weibo_df['Sentiment'][row] = SnowNLP(weibo_df['weibo_text'][row]).sentiments


In [7]:
weibo_df.to_csv('weibo_sen.csv', index=False, encoding='utf-8')

In [8]:
weibo_df.head()

,weibo_id,Label,weibo_text,keyword,user_gender,user_location,user_following,user_follower,weibo_like,weibo_repost,weibo_comment,weibo_date,Sentiment
0,J1zF7smOR,0,这两天，看到首页的几个大咖在讨论“田园女权”，讲真，我不清楚什么是田园女权以前听过，但没深入...,女权,1,广东,1737,1712050,989.0,62.0,115.0,12/5/2020 13:42,1
1,GvXf0ymdJ,0,【好莱坞女星性侵17岁男星，付38万美元封口费，曾举报哈维·韦恩斯坦性侵】《纽约时报》8月1...,NaN,1,北京,343,4314049,539.0,69.0,158.0,23/08/2018 12:40,0.99913
2,J2XeR8rPn,0,“姨味” 怎么讲，挺恶意满满的一个词。形容一个男性“姨”，背后仍然是一种不被察觉的性别歧视和...,厌女,1,浙江,598,850691,255.0,11.0,37.0,21/05/2020 15:33,1
3,IxpfTCMAo,0,控评就没意思了，你们讨厌木大妈不允许有任何不同意见，但事实上我也觉得你们很无聊，针对她哪里不...,"厌女,女权",1,其他,295,4196,1197.0,71.0,100.0,6/3/2020 23:18,1.44761e-07
4,J4B0Mps5Q,0,白人警察可以和黑人兄弟一起携手抗议，但很难想象女权运动时任何肤色的男性会如此这般站在女性这边...,女权,1,其他,427,17319,2121.0,624.0,67.0,1/6/2020 10:39,1


In [36]:
#dv: likeness:8 repost:9 comment:10
#iv:gender:4 sentiment:12 label:1

import itertools as it
dv_list = [8,9,10]
dv_name_dict = {8: 'LikeNum', 9: 'RepostNum', 10: 'CommentNum'}
iv_list = [4,12,1]
iv_name_dict = {4: 'Gender', 12: 'Sentiment', 1: 'Sexist'}
v_comb_list = []
for i in range(1,len(iv_name_dict) + 1):
    for iv_comb in it.combinations(iv_name_dict.keys(), i):
        for dv in dv_name_dict.keys():
            v_comb_list.append([dv, [iv for iv in iv_comb]])

In [41]:
from sklearn import linear_model
from sklearn.feature_selection import f_regression
import math

res_list = []
data = np.array(weibo_df)
data = np.delete(data, 770, axis=0)

for v_comb in v_comb_list:
    X = data[:, v_comb[1]].astype(np.float)
    Y = data[:, v_comb[0]].astype(np.float)
    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    freg=f_regression(X,Y)
    
    coef_list = regr.coef_
    p_list = freg[1].round(3)
    iv_list = [{'name': iv_name_dict[iv_id], 'coef': coef_list[idx].round(3), 'p': p_list[idx]} for idx,iv_id in enumerate(v_comb[1])]
    
    res = {'dv': dv_name_dict[v_comb[0]], 'iv': iv_list, 'intercept': regr.intercept_.round(3), 'r2': regr.score(X, Y).round(3), 'r2_adjusted': (1 - (1-regr.score(X, Y))*(len(Y)-1)/(len(Y)-X.shape[1]-1)).round(5)}
    res_list.append(res)

In [42]:
import json
with open('./reg_results.json', 'w') as f:
    json.dump(res_list, f)

In [67]:
import pandas as pd
# summary statistics of all column

weibo_df['Sentiment'] = pd.to_numeric(weibo_df['Sentiment'])
weibo_df.describe()

,Label,user_gender,user_following,user_follower,weibo_like,weibo_repost,weibo_comment,Sentiment
count,1527.000000,1527.000000,1527.000000,1.527000e+03,1526.000000,1526.000000,1526.000000,1.527000e+03
mean,0.413229,0.652914,594.512115,1.780657e+06,313.944299,37.368938,29.037353,8.228286e-01
std,0.492574,0.476199,701.219580,5.927849e+06,3277.067649,254.544014,96.885550,3.449104e-01
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000,1.776357e-15
25%,0.000000,0.000000,192.500000,2.260000e+02,2.000000,0.000000,3.000000,9.187799e-01
50%,0.000000,1.000000,393.000000,1.870000e+03,10.000000,1.000000,6.000000,9.999771e-01
75%,1.000000,1.000000,735.000000,1.085960e+05,73.000000,13.000000,18.000000,1.000000e+00
max,1.000000,1.000000,10822.000000,7.910873e+07,110632.000000,7888.000000,1723.000000,1.000000e+00


In [68]:
import pandas as pd
# summary statistics of all column


weibo_df.describe(include=['object'])

,weibo_id,weibo_text,keyword,user_location,weibo_date
count,1527,1527,1350,1527,1526
unique,1527,1523,12,36,1518
top,J1zF7smOR,【好莱坞大亨#韦恩斯坦被判刑23年# 超90名女性曾控其性犯罪】3月11日，美国好莱坞制片人...,女权,其他,11/3/2020 23:08
freq,1,3,974,347,2
